In [1]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [2]:
# instalar FFMPEG no windows https://phoenixnap.com/kb/ffmpeg-windows
import ffmpeg

In [3]:
import subprocess, sys, os, json

from datetime import datetime, timedelta

from vosk import Model, KaldiRecognizer

import pprint

#### Write the code

In [4]:
SAMPLE_RATE = 16000
CHUNK_SIZE = 4000

In [5]:
class Transcriber():
    def __init__(self, model_path):
        self.model = Model(model_path)

    def fmt(self, data):
        data = json.loads(data)

        start = min(r["start"] for r in data.get("result", [{ "start": 0 }]))
        end = max(r["end"] for r in data.get("result", [{ "end": 0 }]))

        return {
            "start": str(timedelta(seconds=start)), 
            "end": str(timedelta(seconds=end)), 
            "text": data["text"]
        }

    def transcribe(self, filename):
        rec = KaldiRecognizer(self.model, SAMPLE_RATE)
        rec.SetWords(True)

        if not os.path.exists(filename):
            raise FileNotFoundError(filename)

        transcription = []

        ffmpeg_command = [
                "ffmpeg",
                "-nostdin",
                "-loglevel",
                "quiet",
                "-i",
                filename,
                "-ar",
                str(SAMPLE_RATE),
                "-ac",
                "1",
                "-f",
                "s16le",
                "-",
            ]

        with subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE) as process:

            start_time = datetime.now() 
            while True:
                data = process.stdout.read(4000)
                if len(data) == 0:
                    break
                
                if rec.AcceptWaveform(data):
                    transcription.append(self.fmt(rec.Result()))

            transcription.append(self.fmt(rec.FinalResult()))
            end_time = datetime.now()

            time_elapsed = end_time - start_time
            print(f"Time elapsed  {time_elapsed}")

        return {
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat(),
            "elapsed_time": time_elapsed,
            "transcription": transcription,
        }


In [6]:
def transcribe(filename, model_path):
    transcriber = Transcriber(model_path)
    transcription = transcriber.transcribe(filename)

    pprint.pprint(transcription)

# Transcribe a Portuguese audio

### AUDIO 01 - 1'27

#### Using (large) portuguese language pack model

In [7]:
filename = "audio_01_trecho_livro_ruth_rocha.mp3"
model_path = "vosk-model-small-pt-0.3"

transcribe(filename, model_path)

Time elapsed  0:00:16.529136
{'elapsed_time': datetime.timedelta(seconds=16, microseconds=529136),
 'end_time': '2024-06-03T09:14:24.463392',
 'start_time': '2024-06-03T09:14:07.934256',
 'transcription': [{'end': '0:00:12.480000',
                    'start': '0:00:00.870000',
                    'text': 'olá pessoal tudo bem com vocês hoje vou ler um '
                            'trecho do livro marcelo mello martelo de ruth '
                            'rocha'},
                   {'end': '0:00:18.120000',
                    'start': '0:00:14.190000',
                    'text': 'marcelo vivia fazendo perguntas a todo mundo'},
                   {'end': '0:00:21.749352',
                    'start': '0:00:19.202272',
                    'text': 'pai que é que a chuva cai'},
                   {'end': '0:00:24.599081',
                    'start': '0:00:22.800000',
                    'text': 'em que é que o mar'},
                   {'end': '0:00:28.709714',
                    '

In [8]:
filename = "audio_01_trecho_livro_ruth_rocha.mp3"
# acesso ao full model PT-BR funcionando: https://github.com/alphacep/vosk-api/issues/991
# a versão full disponível no site da Vosk não funciona
model_path = "vosk-model-pt-fb-v0.1.1-pruned"

transcribe(filename, model_path)

Time elapsed  0:00:08.849252
{'elapsed_time': datetime.timedelta(seconds=8, microseconds=849252),
 'end_time': '2024-06-03T09:14:37.353914',
 'start_time': '2024-06-03T09:14:28.504662',
 'transcription': [{'end': '0:00:12.509949',
                    'start': '0:00:00.870000',
                    'text': 'olá pessoal tudo bem com vocês hoje vou ler um '
                            'trecho do livro marcelo marmelo martelo de rute '
                            'rocha'},
                   {'end': '0:00:33.840000',
                    'start': '0:00:14.190000',
                    'text': 'marcelo vivia fazendo perguntas a todo mundo para '
                            'pai porque é que a chuva cai sua mãe por que é '
                            'que o mar não derrama de louvor porque é que o '
                            'cachorro tem quatro pernas as pessoas grandes às '
                            'vezes real'},
                   {'end': '0:00:53.759963',
                    'start': '

### AUDIO 02 - 11'35

In [9]:
filename = "audio_02_trecho_1984_George_Orwell.mp3"
model_path = "vosk-model-small-pt-0.3"

transcribe(filename, model_path)

Time elapsed  0:02:05.358760
{'elapsed_time': datetime.timedelta(seconds=125, microseconds=358760),
 'end_time': '2024-06-03T09:16:43.648685',
 'start_time': '2024-06-03T09:14:38.289925',
 'transcription': [{'end': '0:00:00', 'start': '0:00:00', 'text': ''},
                   {'end': '0:00:18.569832',
                    'start': '0:00:05.910000',
                    'text': 'em onde ficavam estão com curiosidade nos olhos '
                            'mais do que nunca tinha um mestre dedicada um '
                            'aluno peralta mais promissor há um ditado do '
                            'partido que se refere ao controle do passado'},
                   {'end': '0:00:28.470000',
                    'start': '0:00:20.220000',
                    'text': 'repita por favor quem controla o passado controle '
                            'futuro que encontrou do presente controle passado '
                            'repete um instante'},
                   {'end': '0:00:35

In [10]:
filename = "audio_02_trecho_1984_George_Orwell.mp3"
model_path = "vosk-model-pt-fb-v0.1.1-pruned"

transcribe(filename, model_path)

Time elapsed  0:00:59.025347
{'elapsed_time': datetime.timedelta(seconds=59, microseconds=25347),
 'end_time': '2024-06-03T09:17:44.536417',
 'start_time': '2024-06-03T09:16:45.511070',
 'transcription': [{'end': '0:00:00', 'start': '0:00:00', 'text': ''},
                   {'end': '0:00:14.550000',
                    'start': '0:00:05.584671',
                    'text': 'o ficavam estou com curiosidade nos olhos mais do '
                            'que nunca tinham arte um mestre dedicada um aluno '
                            'peralta mais promissor'},
                   {'end': '0:00:19.620000',
                    'start': '0:00:15.270000',
                    'text': 'há um ditado do partido que se refere ao controle '
                            'do passado disse ele'},
                   {'end': '0:00:40.320000',
                    'start': '0:00:20.220271',
                    'text': 'repitam por favor quem controla o passado '
                            'controla o fut